In [15]:
import math
from textblob import TextBlob as tb
import newspaper
import os
import random
import pickle
import import_ipynb
from lexrank_summarizer import summarize
from news_classes import customizedPaper
from news_classes import paragraph

In [16]:
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [17]:
cwd=os.getcwd()+'/'
news_path='news_30_9_2018.pkl'

with open(news_path, 'rb') as input:
    allnews = pickle.load(input)

In [18]:
def getSummarizedFromList(para_list):
    summ_list=[]
    for paragraph in para_list:
        if searched_word.lower() in paragraph.body.lower():
            obj={'title':paragraph.title, 'url':paragraph.url, 'text':summarize(paragraph.body), 'date':paragraph.date, 'brand':paragraph.brand}
            summ_list.append(obj)    
    return summ_list

In [19]:
def getNewsFromKey(searched_word):
    searched_word=searched_word.lower()
    para_list=[]
    for paragraph in allnews.paragraphs:
        if searched_word in paragraph.body.lower():
            if paragraph.date is not None:
                para_list.append(paragraph)
            
    para_list.sort(key=lambda item:(item.date.year, item.date.month, item.date.day), reverse=True)
    
    ret_list=[]
    for i in para_list:
        if searched_word in i.title.lower():
            ret_list.append(i)
    for i in para_list:
        if searched_word not in i.title.lower():
            ret_list.append(i)
    para_list.clear()
    return ret_list

In [20]:
list_of_news=[]
searched_word=""
total_page=0
content_per_page=5

In [21]:
app = Flask(__name__)

In [22]:
@app.route('/show_result', methods=['POST'])
def show_result():
    global searched_word
    searched_word = request.form['news_keyword']
    print(searched_word)
    global list_of_news
    list_of_news=getNewsFromKey(searched_word)
    global total_page
    total_page=math.ceil(len(list_of_news)/content_per_page)
    
    list_ten=[]
    for i in range(min(len(list_of_news), content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [23]:
@app.route('/show_other_page', methods=['POST'])
def show_other_page():
    page_number = int(request.form['page_number'])
        
    list_ten=[]
    for i in range(page_number*content_per_page, min(len(list_of_news), (page_number+1)*content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [24]:
@app.route("/")
def search_key():
    rendered = render_template('ani_search.html')
    #rendered="hello there!"
    return rendered

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('0.0.0.0', 9001, app)

 * Running on http://0.0.0.0:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2018 22:32:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2018 22:32:03] "GET /favicon.ico HTTP/1.1" 404 -


dhaka university


127.0.0.1 - - [30/Oct/2018 22:32:05] "POST /show_result HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2018 22:32:09] "POST /show_other_page HTTP/1.1" 200 -
